<a href="https://colab.research.google.com/github/icebearian/CS115-Mathematics-for-Computer-Science/blob/master/SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>